# 📥 Torrent → Google Drive Downloader (Enhanced v5.0)

**Improvements in this version:**
- ✅ Proper session cleanup (prevents memory leaks)
- ✅ Resume capability for interrupted downloads
- ✅ Better error handling and recovery
- ✅ Enhanced progress tracking with ETA
- ✅ Keyboard interrupt handling with resume data saving
- ✅ Safer settings_pack fallback logic

This notebook uses modern libtorrent 2.x API with robust error handling.

⚠️ Only download content you are legally allowed to download.

**Tips:**
- If Drive mount fails, files save to `/content/torrents/`
- Public trackers are injected by default for better peer discovery
- Use `resume_file` parameter to continue interrupted downloads


In [ ]:
# Mount Google Drive (optional, robust fallback)
import os

drive_mounted = False
IN_COLAB = False
try:
    from google.colab import drive as _colab_drive
    IN_COLAB = True
except Exception:
    _colab_drive = None
    IN_COLAB = False

try:
    if IN_COLAB:
        if not os.path.exists('/content/drive/MyDrive'):
            _colab_drive.mount('/content/drive', force_remount=True)
        drive_mounted = os.path.exists('/content/drive/MyDrive')
        if drive_mounted:
            print('✅ Google Drive mounted: /content/drive/MyDrive')
except Exception as e:
    print('⚠️ Google Drive mount failed:', e)
    drive_mounted = False

if not drive_mounted:
    os.makedirs('/content/torrents', exist_ok=True)
    print('Using local path: /content/torrents')
else:
    # create a default folder inside MyDrive for convenience
    default_gdrive_path = '/content/drive/MyDrive/Torrent'
    os.makedirs(default_gdrive_path, exist_ok=True)
    print('Using Google Drive path:', default_gdrive_path)


In [ ]:
# Install libtorrent (2.0.11) and requests if missing
import sys
import subprocess
import importlib

def pip_install(pkg):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])
    except subprocess.CalledProcessError as e:
        print(f'pip install failed for {pkg}:', e)
        raise

lt = None
try:
    import libtorrent as lt
    print('✅ libtorrent present:', getattr(lt, 'version', getattr(lt, '__version__', 'unknown')))
except Exception:
    print('⚠️ libtorrent not found. Trying libtorrent==2.0.11 ...')
    try:
        pip_install('libtorrent==2.0.11')
        importlib.invalidate_caches()
        import libtorrent as lt
        print('✅ libtorrent installed:', getattr(lt, 'version', getattr(lt, '__version__', 'unknown')))
    except Exception as e:
        print('Primary install failed, trying alternative wheel name py3-libtorrent...')
        pip_install('py3-libtorrent')
        importlib.invalidate_caches()
        import libtorrent as lt
        print('✅ libtorrent installed via py3-libtorrent:', getattr(lt, 'version', getattr(lt, '__version__', 'unknown')))

try:
    import requests
except Exception:
    print('Installing requests...')
    pip_install('requests')
    importlib.invalidate_caches()
    import requests
    print('✅ requests installed')


In [ ]:
# Enhanced torrent engine with proper cleanup and resume support
import libtorrent as lt
import time
import os
import urllib.parse
import math
import shutil
import traceback

def _create_session(settings: dict):
    """Try lt.session(settings) first; fallback to settings_pack + apply_settings.
    Enhanced with safer attribute access.
    """
    try:
        ses = lt.session(settings)
        return ses
    except TypeError:
        # Fallback to settings_pack for older bindings
        sp = lt.settings_pack()
        for k, v in settings.items():
            try:
                # Try direct attribute access first
                if hasattr(lt.settings_pack, k):
                    key = getattr(lt.settings_pack, k)
                # Try setting_by_name if available
                elif hasattr(lt.settings_pack, 'setting_by_name'):
                    key = lt.settings_pack.setting_by_name(k)
                else:
                    # Skip unknown settings
                    continue
                
                # Set value based on type
                if isinstance(v, bool):
                    sp.set_bool(key, v)
                elif isinstance(v, int):
                    sp.set_int(key, v)
                else:
                    sp.set_str(key, str(v))
            except (AttributeError, TypeError, ValueError) as e:
                # Silently skip invalid settings
                pass
        
        ses = lt.session()
        ses.apply_settings(sp)
        return ses

def _add_trackers_to_magnet(magnet: str, trackers: list) -> str:
    """Append public trackers to magnet link for better peer discovery."""
    if not magnet.startswith('magnet:'):
        return magnet
    for tr in trackers:
        magnet += '&tr=' + urllib.parse.quote(tr)
    return magnet

def _fmt_eta(remaining_bytes, rate):
    """Format estimated time of arrival."""
    if rate <= 0: 
        return 'ETA: ∞'
    secs = int(remaining_bytes / rate)
    m, s = divmod(secs, 60)
    h, m = divmod(m, 60)
    if h: 
        return f'ETA: {h}h {m}m'
    if m: 
        return f'ETA: {m}m {s}s'
    return f'ETA: {s}s'

def _fmt_size(bytes_val):
    """Format bytes into human-readable size."""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_val < 1024.0:
            return f"{bytes_val:.2f} {unit}"
        bytes_val /= 1024.0
    return f"{bytes_val:.2f} PB"

def download_colab(magnet_link: str, save_path: str,
                   max_peers: int = 400, 
                   peer_connect_timeout: int = 3,
                   add_trackers: bool = True,
                   stall_timeout: int = 900,
                   auto_zip: bool = False, 
                   zip_name: str = None,
                   resume_file: str = None):
    """Download a magnet to save_path using modern libtorrent 2.x patterns.
    
    Args:
        magnet_link: Magnet URI to download
        save_path: Directory to save downloaded files
        max_peers: Maximum number of peer connections (default: 400)
        peer_connect_timeout: Timeout for peer connections in seconds (default: 3)
        add_trackers: Whether to inject public trackers (default: True)
        stall_timeout: Seconds before considering download stalled (default: 900)
        auto_zip: Whether to zip downloaded content (default: False)
        zip_name: Custom name for zip file (default: uses torrent name)
        resume_file: Path to save/load resume data for continuation (default: None)
    
    Returns:
        bool: True if download completed successfully, False otherwise
    """
    ses = None
    h = None
    
    try:
        # Optional public trackers (helps tracker-less magnets)
        if add_trackers:
            trackers = [
                'udp://tracker.opentrackr.org:1337/announce',
                'udp://tracker.torrent.eu.org:451/announce',
                'udp://exodus.desync.com:6969/announce',
                'udp://tracker.openbittorrent.com:6969/announce',
                'udp://open.stealth.si:80/announce',
                'udp://tracker.tiny-vps.com:6969/announce'
            ]
            magnet_link = _add_trackers_to_magnet(magnet_link, trackers)

        # Configure session settings
        settings = {
            'user_agent': f'libtorrent/{getattr(lt, "version", "unknown")}',
            'listen_interfaces': '0.0.0.0:6881',
            'enable_dht': True,
            'enable_lsd': True,
            'announce_to_all_trackers': True,
            'announce_to_all_tiers': True,
            'connections_limit': max_peers,
            'max_peerlist_size': max_peers * 2,
            'peer_connect_timeout': peer_connect_timeout,
            'request_timeout': 10,
            'min_reconnect_time': 5,
            'active_downloads': -1,
            'active_seeds': -1,
            'allow_multiple_connections_per_ip': True,
        }
        ses = _create_session(settings)

        # Parse magnet and configure download
        params = lt.parse_magnet_uri(magnet_link)
        params.save_path = save_path
        params.flags |= lt.torrent_flags.sequential_download
        
        # Load resume data if available
        if resume_file and os.path.exists(resume_file):
            try:
                with open(resume_file, 'rb') as f:
                    resume_data = f.read()
                    if resume_data:
                        params.resume_data = resume_data
                        print('📥 Loaded resume data from', resume_file)
            except Exception as e:
                print(f'⚠️ Failed to load resume data: {e}')

        h = ses.add_torrent(params)
        print('🔗 Magnet added')
        print('📍 Save path:', save_path)

        # Wait for metadata with timeout
        meta_wait = 0
        while not h.status().has_metadata:
            print('📡 Fetching metadata...', end='\r')
            time.sleep(1)
            meta_wait += 1
            if meta_wait > 120:
                print('\n⚠️ Metadata fetch taking too long; continuing anyway')
                break

        s = h.status()
        if s.has_metadata:
            print(f'\n✅ Metadata fetched — Name: {s.name}')
            print(f'📦 Size: {_fmt_size(s.total_wanted)}')
        else:
            print(f'\n▶️ Starting without metadata')

        # Progress loop with stall detection
        last_progress = 0.0
        last_change_ts = time.time()
        
        while not h.status().is_seeding:
            s = h.status()
            total = s.total_wanted if s.total_wanted > 0 else 0
            done = s.total_wanted_done
            remaining = max(0, total - done)
            eta = _fmt_eta(remaining, s.download_rate) if total else 'ETA: ?'
            
            # Enhanced progress display
            progress_bar = '█' * int(s.progress * 20) + '░' * (20 - int(s.progress * 20))
            print(f"\r{progress_bar} {s.progress*100:5.2f}% | "
                  f"↓{s.download_rate/1000:6.1f} kB/s | "
                  f"↑{s.upload_rate/1000:6.1f} kB/s | "
                  f"peers {s.num_peers} | {eta}      ", end='')
            
            # Stall detection
            if s.progress - last_progress > 0.001:
                last_progress = s.progress
                last_change_ts = time.time()
            elif stall_timeout and stall_timeout > 0:
                if (time.time() - last_change_ts > stall_timeout):
                    print('\n⏱️ Download appears stalled. Stopping.')
                    break
            
            time.sleep(5)

        completed = h.status().is_seeding
        print()  # New line after progress
        
        if completed:
            print('🎉 COMPLETE — saved to', save_path)
        else:
            print('⚠️ Stopped before completion.')
            # Save resume data for later continuation
            if resume_file:
                try:
                    resume_data = h.save_resume_data()
                    with open(resume_file, 'wb') as f:
                        f.write(lt.bencode(resume_data))
                    print(f'💾 Resume data saved to {resume_file}')
                    print('💡 Use the same resume_file to continue this download later')
                except Exception as e:
                    print(f'⚠️ Failed to save resume data: {e}')

        # Optional zip with error handling
        if auto_zip and completed:
            try:
                s = h.status()
                name = s.name or 'torrent'
                base = (zip_name or name).replace(' ', '_')
                target_path = os.path.join(save_path, name)
                
                if os.path.isdir(target_path):
                    root_dir = target_path
                else:
                    root_dir = save_path
                
                zip_target = os.path.join(save_path, base)
                print(f'🗜️ Creating zip: {zip_target}.zip ...')
                shutil.make_archive(zip_target, 'zip', root_dir)
                
                # Verify zip was created
                zip_path = zip_target + '.zip'
                if os.path.exists(zip_path):
                    zip_size = os.path.getsize(zip_path)
                    print(f'✅ Zip created: {zip_path} ({_fmt_size(zip_size)})')
                else:
                    print('⚠️ Zip file not found after creation')
            except Exception as e:
                print(f'❌ Zip creation failed: {e}')
                traceback.print_exc()
        
        return completed
        
    except KeyboardInterrupt:
        print('\n⚠️ Interrupted by user')
        # Save resume data on interrupt
        if resume_file and h:
            try:
                resume_data = h.save_resume_data()
                with open(resume_file, 'wb') as f:
                    f.write(lt.bencode(resume_data))
                print(f'💾 Resume data saved to {resume_file}')
                print('💡 Re-run with the same resume_file to continue')
            except Exception as e:
                print(f'⚠️ Failed to save resume data: {e}')
        raise
        
    except Exception as e:
        print(f'\n❌ Download failed: {e}')
        traceback.print_exc()
        return False
        
    finally:
        # Proper cleanup to prevent memory leaks
        if h:
            try:
                h.pause()
                if ses:
                    ses.remove_torrent(h)
            except Exception as e:
                print(f'⚠️ Cleanup warning: {e}')
        if ses:
            try:
                del ses
            except Exception:
                pass

print('✅ Enhanced torrent engine loaded')


## Usage Examples

Run the cells below to download torrents. Customize the parameters as needed.

In [ ]:
# Example 1: Basic download
# Replace with your magnet link
magnet = 'magnet:?xt=urn:btih:YOUR_HASH_HERE'

# Set save path (use Google Drive if mounted, otherwise local)
if drive_mounted:
    save_to = '/content/drive/MyDrive/Torrent'
else:
    save_to = '/content/torrents'

# Download
success = download_colab(
    magnet_link=magnet,
    save_path=save_to,
    add_trackers=True,
    stall_timeout=900  # 15 minutes
)

if success:
    print('✅ Download successful!')
else:
    print('❌ Download failed or incomplete')


In [ ]:
# Example 2: Download with resume capability
magnet = 'magnet:?xt=urn:btih:YOUR_HASH_HERE'

if drive_mounted:
    save_to = '/content/drive/MyDrive/Torrent'
    resume_data_file = '/content/drive/MyDrive/Torrent/.resume_data'
else:
    save_to = '/content/torrents'
    resume_data_file = '/content/torrents/.resume_data'

# This will save resume data if interrupted
# Run again with same resume_file to continue
success = download_colab(
    magnet_link=magnet,
    save_path=save_to,
    resume_file=resume_data_file,
    add_trackers=True
)


In [ ]:
# Example 3: Download with auto-zip
magnet = 'magnet:?xt=urn:btih:YOUR_HASH_HERE'

if drive_mounted:
    save_to = '/content/drive/MyDrive/Torrent'
else:
    save_to = '/content/torrents'

success = download_colab(
    magnet_link=magnet,
    save_path=save_to,
    auto_zip=True,
    zip_name='my_download',
    add_trackers=True
)


In [ ]:
# Example 4: Advanced configuration
magnet = 'magnet:?xt=urn:btih:YOUR_HASH_HERE'

if drive_mounted:
    save_to = '/content/drive/MyDrive/Torrent'
else:
    save_to = '/content/torrents'

success = download_colab(
    magnet_link=magnet,
    save_path=save_to,
    max_peers=600,              # More connections
    peer_connect_timeout=5,     # Longer timeout
    stall_timeout=1800,         # 30 min stall timeout
    add_trackers=True,
    auto_zip=False
)
